Networks - Final Project

"Is there an optimal road to close and convert to a public use space in Barcelona that minimizes the impact on traffic?"

-Clarice Mottet, Amber Walker, Mox Ballo

Outline:
-import Barcelona road network data into programming environment
-subset the data to be only a neighborhood of Barcelona
-determine alpha and beta parameters for latency function
-create a dataset that is a very simple road network
-write a function that calculates latency between nodes



https://nbviewer.org/github/PyTrans/Urban-Network-Analysis/blob/master/Trip_Assignment-Frank-Wolfe_Algorithm.ipynb

In [4]:
#libraries

import pandas as pd
import numpy as np
import networkx as nx
import networkx as nx
import scipy.integrate as integrate 
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
from scipy.misc import derivative
import TransportationNetworks as tn

path_in_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/inputs'
path_out_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/outputs'

In [ ]:
#functions

#from website to calculate latency
def BPR(t0, xa, ca, alpha, beta):
    ta = t0*(1+alpha*(xa/ca)**beta)
    return ta

#from website to calculate nash equilibrium (when SO = False)
def calculateZ(theta, network, SO):
    z = 0
    for linkKey, linkVal in network.items():
        t0 = linkVal['t0']
        ca = linkVal['capa']
        beta = linkVal['beta']
        alpha = linkVal['alpha']
        aux = linkVal['auxiliary'][-1]
        flow = linkVal['flow'][-1]
        
        if SO == False:
            z += integrate.quad(lambda x: BPR(t0, x, ca, alpha, beta), 0, flow+theta*(aux-flow))[0]
        elif SO == True:
            z += list(map(lambda x : x * BPR(t0, x, ca, alpha, beta), [flow+theta*(aux-flow)]))[0]
    return z

def lineSearch(network, SO):
    theta = minimize_scalar(lambda x: calculateZ(x, network, SO), bounds = (0,1), method = 'Bounded')
    return theta.x

In [ ]:
directory = ".\\Data\\TransportationNetworks\\SiouxFalls\\"
link_file = '{}SiouxFalls_net.tntp'.format(directory)
trip_file = '{}SiouxFalls_trips.tntp'.format(directory)
node_file = '{}SiouxFalls_node.tntp'.format(directory)

In [ ]:
SO = False # True - System optimum, False - User equilibrium
siouxFalls = tn.Network(link_file, trip_file, node_file, SO)